# Working with TrackMate data

In [1]:
import pycellin as pc

# %matplotlib widget

## 0. Example data

In [2]:
# Main example.
trackmate_xml = "../sample_data/Ecoli_growth_on_agar_pad.xml"

# Other examples.
trackmate_xml_fusions = "../sample_data/Ecoli_growth_on_agar_pad_with_fusions.xml"

## 1. Creating a model from a TrackMate XML file

In [3]:
# Parse the XML file and create a Pycellin Model object
# that contains all the data from the XML file.
model = pc.load_TrackMate_XML(trackmate_xml)

In Pycellin, tracks from TrackMate are modeled by directed acyclic graphs and are called cell lineages. **It means that splitting events are allowed, they even are recommended if you want to take full advantage of Pycellin. However, MERGING EVENTS ARE NOT SUPPORTED.** If you try to use Pycellin on a lineage with merging events, it may crash or produce incorrect results, especially if you are computing features related to tracking.


In [4]:
model_fusions = pc.load_TrackMate_XML(trackmate_xml_fusions)

/mnt/data/Code/pycellin/pycellin/pycellin/io/trackmate/loader.py:1233: UserWarning: Unsupported data, 3 cell fusions detected. It is advised to deal with them before any other processing, especially for tracking related features. Crashes and incorrect results can occur. See documentation for more details.
  warnings.warn(fusion_warning)


`load_TrackMate_XML()` automatically check for fusion when loading a file, and will warn the user if any are found. It is then up to the user to decide what to do. **Be careful if you decide to proceed without first sorting out the fusions.**

Pycellin is creating one lineage per TrackMate track. \
The arguments `keep_all_spots` and `keep_all_tracks` allow you to keep (value to `True`) or discard (value to `False`) spots/tracks that were filtered out in TrackMate. These arguments are false by default.

We can display basic information about the model.

In [5]:
print(model)
print(f"This model contains {model.data.number_of_lineages()} lineages:")
for lin_ID, lineage in model.data.cell_data.items():
    print(f"- ID {lin_ID}: {lineage}")

Model named 'Ecoli_growth_on_agar_pad' with 3 lineages, built from TrackMate.
This model contains 3 lineages:
- ID 0: CellLineage of ID 0 with 152 cells and 151 links.
- ID 1: CellLineage of ID 1 with 189 cells and 188 links.
- ID 2: CellLineage of ID 2 with 185 cells and 184 links.


So here we have three different cell lineages that you can identify by their ID: 0, 1 and 2.

In [13]:
import networkx as nx
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from cmap import Colormap

In [21]:
cm = Colormap('glasbey:glasbey').to_plotly()
cm = [rgb for (value, rgb) in cm[1:]]

In [7]:
def plot_cell_trajectory(
    lineage: pc.CellLineage,
    y_feature: str,
    x_feature: str = "frame",
    start_noi: int | None = None,
    end_noi: int | list[int] | None = None,
    show_divisions: bool = True,
    color_palette: list[str] = px.colors.qualitative.Alphabet,
) -> go.Figure:
    if start_noi is None:
        start_noi = lineage.get_root()
        # TODO: deal with the case where there are multiple roots.
    if end_noi is None:
        end_noi = lineage.get_leaves()

    if isinstance(end_noi, int):
        trajs = [nx.shortest_path(lineage, start_noi, end_noi)]
    elif isinstance(end_noi, list):
        trajs = [nx.shortest_path(lineage, start_noi, end) for end in end_noi]

    fig = go.Figure()
    for i, traj in enumerate(trajs):
        # Cycle through the color palette.
        color = color_palette[i % len(color_palette)]
        if show_divisions:
            # For grouping the legend by trajectory.
            traj_name = "Trajectory"
            legend_group = f"{traj[0]}-{traj[-1]}"
        else:
            traj_name = f"{traj[0]}-{traj[-1]}"
            legend_group = None

        # Plot the trajectory.
        fig.add_trace(
            go.Scatter(
                x=[lineage.nodes[node][x_feature] for node in traj],
                y=[lineage.nodes[node][y_feature] for node in traj],
                mode="lines",
                line=dict(color=color),
                name=traj_name,
                legendgroup=legend_group,
                legendgrouptitle=dict(text=legend_group),
            )
        )

        # Plot the divisions.
        if show_divisions:
            divs = lineage.get_divisions()
            div_nodes = [node for node in traj if node in divs]
            fig.add_trace(
                go.Scatter(
                    x=[lineage.nodes[node][x_feature] for node in div_nodes],
                    y=[lineage.nodes[node][y_feature] for node in div_nodes],
                    mode="markers",
                    marker=dict(color=color),
                    name="Divisions",
                    legendgroup=legend_group,
                )
            )

    return fig

In [8]:
print(px.colors.qualitative.Plotly)

['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']


In [9]:
lin0 = model.data.cell_data[0]
lin0.plot_tree()

In [22]:
fig = plot_cell_trajectory(
    lin0,
    "AREA",
    # start_noi=9051,
    # end_noi=[9470, 9476],
    # end_noi=9470,
    # show_divisions=False,
    color_palette=cm,
)
fig.show()

*lossless*

## 2. Accessing, computing and adding features

### 2.1 TrackMate features

### 2.2 Pycellin features

### 2.3 Custom features

## 3. Exporting the data

### 3.1 Back to TrackMate

### 3.2 In a CSV format